In [459]:
import pdfplumber
import pandas as pd


def extract_pdf_text(pdf_path):
    # Open the PDF file
    with pdfplumber.open(pdf_path) as pdf:
        # Initialize an empty list to store the extracted data
        data = []
        # Iterate through each page
        for page_num, page in enumerate(pdf.pages):
            # Extract the text with bounding boxes
            for element in page.extract_words():
                text = element['text']
                x0, y0, x1, y1 = element['x0'], element['top'], element['x1'], element['bottom']
                data.append([page_num + 1, text, x0, y0, x1, y1])

        # Convert the list to a DataFrame
        df = pd.DataFrame(data, columns=['page', 'text', 'x0', 'y0', 'x1', 'y1'])
        return df


def find_interval(number, intervals):
    intervals = sorted(intervals)
    for interval in intervals:
        if number >= interval[0] and number < interval[1]:
            return interval
    return None


def assign_intervals_and_values(df, gridlines):
    df["interval"] = df['x0'].apply(lambda x: find_interval(x, list(gridlines.keys())))
    df["value"] = df["interval"].apply(lambda x: gridlines.get(x, None) if x else None)
    return df


def process_consecutive_values(df, target_value):
    processed_rows = []
    current_row = None

    for _, row in df.iterrows():
        if row['value'] == target_value:
            if current_row is None:
                current_row = row.copy()
            else:
                # Concatenate the text field
                current_row['text'] += row['text']
                # Update the bounding box
                current_row['x1'] = max(current_row['x1'], row['x1'])
                current_row['y1'] = max(current_row['y1'], row['y1'])
        else:
            if current_row is not None:
                processed_rows.append(current_row)
                current_row = None
            processed_rows.append(row)

    if current_row is not None:
        processed_rows.append(current_row)

    return pd.DataFrame(processed_rows)


In [460]:
def update_identifier_ranges(df, unique_identifier):
    df_identifier = df[df["value"] == unique_identifier].sort_values(by=["page","y1"]).reset_index(drop=True)
    for index in df_identifier.index:
        if index == 0:
            df_identifier.loc[index, "identifier_range_y0"] = df_identifier.loc[index, 'y0'] 
            df_identifier.loc[index, "identifier_range_y1"] = df_identifier.loc[index, 'y1']
        elif index == df_identifier.index.max():
            df_identifier.loc[index, "identifier_range_y0"] = df_identifier.loc[index-1, 'y1'] 
            df_identifier.loc[index, "identifier_range_y1"] = df_identifier.loc[index, 'y1']
        else:
            df_identifier.loc[index, "identifier_range_y0"] = df_identifier.loc[index-1, 'y1']
            df_identifier.loc[index, "identifier_range_y1"] = df_identifier.loc[index, 'y1']

    df_identifier = df_identifier[["page", "text", "identifier_range_y0", "identifier_range_y1"]]
    return df_identifier

In [461]:
def concatenate_values(df):
    new_value_column = []
    new_text_column = []

    # Iterate over the dataframe to concatenate values
    current_value = None
    current_text = ""

    for value, text in zip(df['value'], df['text']):
        if value == current_value:
            current_text += text + " "
        else:
            if current_value is not None:
                new_value_column.append(current_value)
                new_text_column.append(current_text.strip())
            current_value = value
            current_text = text + " "  # Added space at the end of each text

    # Append the last accumulated values
    if current_value is not None:
        new_value_column.append(current_value)
        new_text_column.append(current_text.strip())

    # Create a new DataFrame with the concatenated values
    new_df = pd.DataFrame({
        'value': new_value_column,
        'text': new_text_column
    })
    return new_df

def transform_df(new_df, unique_identifier):
    new_df[unique_identifier] = new_df.apply(lambda x: x.text if x.value == unique_identifier else None, axis=1).ffill()
    new_df = new_df.pivot_table(index=unique_identifier, columns='value', values='text', aggfunc='first')
    return new_df

In [462]:
council = "Wiltshire"
config = {
    "Bedford": {
        "gridlines": {
            (52, 121): "plate",
            (121, 220): "vrm",
            (220, 312): "make",
            (312, 418): "model",
            (418, 482): "licence_start",
            (482, 538): "licence_end"
        },
        "unique_identifier": "plate"
    },
    "BCP": {
        "gridlines": {
            (42, 76): "reference_number",
            (76, 212): "name",
            (212, 268): "vrm",
            (268, 311): "licence_number",
            (311, 364): "licence_start",
            (364, 412): "licence_end",
            (412, 528): "make",
            (528, 590): "licence_type",
        },
        "unique_identifier": "reference_number"
    },
    "Cheshire West": {
        "gridlines": {
            (52, 160): "make",
            (160, 478): "model",
            (478, 540): "vrm"
        },
        "unique_identifier": "vrm"
    },
    "East Northamptonshire": {
        "gridlines": {
            (55, 130): "vrm",
            (160, 240): "make",
            (240, 540): "model"
        },
        "unique_identifier": "vrm"
    },
    "Fylde": {
        "gridlines": {
            (18, 87): "vrm",
            (87, 216): "make",
            (216, 340): "model",
            (340, 460): "licence_start",
            (460, 540): "licence_end"
        },
        "unique_identifier": "vrm"
    },
    "Great Yarmouth": {
        "gridlines": {
            (52, 160): "vrm",
            (160, 238): "make",
            (238, 540): "model",
        },
        "unique_identifier": "vrm"
    },
    "Horsham": {
        "gridlines": {
            (52, 103): "reference_number",
            (103, 152): "licence_type",
            (152, 218): "name",
            (218, 266): "vrm",
            (266, 320): "make",
            (320, 386): "model",
            (386, 446): "licence_start",
            (446, 484): "licence_end",
            (484, 540): "plate",
        },
        "unique_identifier": "vrm"
    },
    "Kettering": {
        "gridlines": {
            (55, 102): "vrm",
            (148, 214): "make",
            (214, 540): "model"
        },
        "unique_identifier": "vrm"
    },
    "Maldon": {
        "gridlines": {
            (35, 142): "plate",
            (142, 258): "vrm",
            (258, 312): "make",
            (312, 445): "model",
            (445, 576): "colour",
            (576, 648): "seats",
            (648, 736): "licence_start",
            (736, 800): "licence_end",
        },
        "unique_identifier": "vrm"
    },
    "Milton Keynes": {
        "gridlines": {
            (256, 288): "plate",
            (288, 336): "vrm",
            (336, 406): "make",
            (406, 488): "model",
            (488, 742): "name",
            (742, 800): "licence_end"
        },
        "unique_identifier": "plate"
    },
    "North Lanarkshire": {
        "gridlines": {
            (52, 140): "vrm",
            (140, 210): "make",
            (210, 360): "model",
        },
        "unique_identifier": "vrm"
    },
    "Powys": {
        "gridlines": {
            (3, 38): "plate",
            (98, 222): "vrm",
            (222, 312): "make",
            (312, 404): "model",
            (404, 482): "licence_start",
        },
        "unique_identifier": "plate"
    },
    "Swansea": {
        "gridlines": {
            (52, 130): "vrm",
            (130, 240): "make"
        },
        "unique_identifier": "vrm"
    },
    "Wealden": {
        "gridlines": {
            (56, 98): "plate",
            (98, 136): "vrm",
            (136, 206): "make",
            (206, 272): "model",
            (272, 315): "colour",
            (315, 404): "name",
            (450, 488): "licence_start",
            (496, 540): "licence_end",
        },
        "unique_identifier": "vrm"
    },
    "West Lothian": {
        "gridlines": {
            (52, 184): "licence_type",
            (184, 295): "vrm",
            (295, 372): "make",
            (372, 440): "model",
        },
        "unique_identifier": "vrm"
    },
    "West Morland Barrow": {
        "gridlines": {
            (55, 118): "vrm",
            (118, 200): "make",
            (200, 300): "model",
        },
        "unique_identifier": "vrm"
    },
    "Argyll and Bute": {
        "gridlines": {
            (52, 158): "vrm",
            (158, 245): "make",
            (245, 400): "model",
        },
        "unique_identifier": "vrm"
    },
    "East Lothian": {
        "gridlines": {
            (77, 170): "vrm",
            (220, 312): "make",
            (312, 418): "model",
        },
        "unique_identifier": "vrm"
    },
    "Renfrewshire": {
        "gridlines": {
            (52, 138): "reference_number",
            (138, 248): "vrm",
            (248, 440): "make",
        },
        "unique_identifier": "vrm"
    },
    "Salford": {
        "gridlines": {
            (28, 78): "reference_number",
            (78, 136): "licence_type",
            (136, 198): "licence_start",
            (198, 250): "licence_end",
            (286, 392): "make",
            (392, 450): "vrm",
        },
        "unique_identifier": "reference_number"
    },
    "Corby": {
        "gridlines": {
            (248, 312): "plate",
            (312, 390): "licence_start",
            (390, 460): "vrm",
            (460, 538): "make",
            (538, 600): "model",
        },
        "unique_identifier": "plate"
    },
    "Cornwall": {
        "gridlines": {
            (28, 121): "reference_number",
            (208, 318): "vrm",
            (318, 416): "make",
            (416, 570): "model",
            (618, 700): "licence_end",
        },
        "unique_identifier": "reference_number"
    },
    "Derby": {
        "gridlines": {
            (32, 88): "plate",
            (88, 320): "name",
            (320, 416): "vrm",
            (416, 540): "make",
        },
        "unique_identifier": "plate"
    },
    "East Dunbarton": {
        "gridlines": {
            (76, 150): "vrm",
            (150, 240): "make",
            (240, 400): "model",
        },
        "unique_identifier": "vrm"
    },
    "Eastleigh": {
        "gridlines": {
            (56, 196): "vrm",
            (196, 276): "make",
            (276, 400): "model",
        },
        "unique_identifier": "vrm"
    },
    "Anglesey": {
        "gridlines": {
            (52, 121): "licence_type",
            (121, 220): "vrm",
            (220, 312): "make",
            (312, 418): "model",
        },
        "unique_identifier": "vrm"
    },
    "Perth Kinross": {
        "gridlines": {
            (76, 196): "vrm",
            (196, 286): "make",
            (286, 400): "model",
        },
        "unique_identifier": "vrm"
    },
    "Rother": {
        "gridlines": {
            (50, 96): "plate",
            (96, 148): "vrm",
            (148, 212): "make",
            (212, 268): "model",
            (456, 499): "licence_start",
            (499, 560): "licence_end",
        },
        "unique_identifier": "plate"
    },
    "Stockton": {
        "gridlines": {
            (40, 230): "licence_type",
            (276, 386): "vrm",
            (386, 510): "make",
            (510, 578): "licence_start",
            (578, 644): "licence_end",
        },
        "unique_identifier": "vrm"
    },
    "Stroud": {
        "gridlines": {
            (50, 128): "vrm",
            (128, 206): "make",
            (206, 340): "model",
            (340, 420): "licence_start",
            (420, 540): "licence_end",
        },
        "unique_identifier": "vrm"
    },
    "Wellingborough": {
        "gridlines": {
            (50, 128): "vrm",
            (128, 206): "make",
            (206, 340): "model",
        },
        "unique_identifier": "vrm"
    },
    "West Lancashire": {
        "gridlines": {
            (20, 50): "plate",
            (50, 116): "vrm",
            (116, 286): "licence_type",
            (286, 360): "make",
            (360, 510): "model",
            (703, 760): "licence_start",
            (760, 826): "licence_end",
        },
        "unique_identifier": "plate"
    },
    "Wigan": {
        "gridlines": {
            (76, 200): "licence_type",
            (200, 324): "plate",
            (324, 436): "vrm",
            (436, 538): "make",
            (538, 654): "model",
            (1110, 1200): "licence_end",
        },
        "unique_identifier": "plate"
    },
    "Wiltshire": {
        "gridlines": {
            (2, 82): "plate",
            (128, 188): "make",
            (188, 272): "model",
            (272, 356): "vrm",
            (356, 428): "licence_start",
            (428, 540): "licence_end",
        },
        "unique_identifier": "plate"
    }
}

gridlines = config[council]["gridlines"]
unique_identifier = config[council]["unique_identifier"]

pdf_path = f'pdfs/tabular/{council}.pdf'
df = extract_pdf_text(pdf_path)


In [463]:
df.head(50)

,page,text,x0,y0,x1,y1
0,1,Plate,3.36000,2.0916,27.12456,12.0516
1,1,Number,29.89344,2.0916,67.59204,12.0516
2,1,Colour,83.06400,2.0916,115.23480,12.0516
3,1,Make,129.62000,2.0916,154.69928,12.0516
4,1,Model,190.37000,2.0916,219.39344,12.0516
5,1,Registration,274.01000,2.0916,332.15648,12.0516
6,1,Issue,345.79000,2.0916,371.23780,12.0516
7,1,Date,374.00668,2.0916,395.58004,12.0516
8,1,Expiry,412.03000,2.0916,442.43788,12.0516
9,1,Date,445.03744,2.0916,466.62076,12.0516


In [464]:
df[100:150]

,page,text,x0,y0,x1,y1
100,1,Silver,83.06400,150.4316,107.76480,160.3916
101,1,Ford,129.62000,150.4316,150.11768,160.3916
102,1,Transit,190.37000,150.4316,220.94720,160.3916
103,1,SF70,274.01000,150.4316,297.79448,160.3916
104,1,EVT,300.52352,150.4316,319.80608,160.3916
105,1,02/11/2022,358.51000,150.4316,408.18964,160.3916
106,1,01/11/2023,429.43000,150.4316,479.10964,160.3916
107,1,HCV030,3.36000,162.7916,40.95900,172.7516
108,1,Silver,83.06400,162.7916,107.76480,172.7516
109,1,Seat,129.62000,162.7916,150.02804,172.7516


In [465]:
df = assign_intervals_and_values(df, gridlines)
df = process_consecutive_values(df, target_value=unique_identifier)
df_identifier = update_identifier_ranges(df, unique_identifier)
df_reduced = df[["text", "value"]].reset_index(drop=True)
new_df = concatenate_values(df_reduced)
new_df = transform_df(new_df, unique_identifier)

In [466]:
new_df

value,licence_end,licence_start,make,model,plate,vrm
plate,,,,,,
3381,02/11/2023,03/11/2022,Volkswagen,Sharan,3381,GK20 FXF
3543,05/02/2024,06/02/2023,Skoda,Karoq,3543,WV68 EPO
3562,06/03/2024,07/03/2023,Skoda,Octavia,3562,WV68 EUK
HCV,30/11/2023,01/12/2022,Skoda,Superb,HCV,R10 SFB
HCV001,16/04/2023,17/04/2022,Skoda,Octavia,HCV001,WN65 SXA
...,...,...,...,...,...,...
N2140,21/08/2023,10/03/2023,Vauxhall,Astra,N2140,FG14 XLS
N2434,07/02/2024,08/02/2023,Toyota,Avensis,N2434,FG15 WCL
N3235,09/05/2023,10/05/2022,Toyota,Prius,N3235,RK69 LFL


In [467]:
new_df.to_csv('inspection.csv')